# self-editing memory from scratch (WITHOUT LETTA)

In [1]:
# we need a .env file in the same directory as this code for this to work

from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()  # load .env as current environment for os to detect

# create OpenAI client
client = OpenAI(
    api_key=str(os.getenv("OPENAI_API_KEY"))
)

model = "gpt-4o-mini"

## understanding the LLM context window

### no memory 🥺

**note:** the roles `system`, `user` and `assistant`are keywords expected by OpenAI. 

In [2]:
system_prompt = "You are a chatbot."

# make a chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt},
        
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"},
    ]
)

chat_completion.choices[0].message.content

"I'm sorry, but I don't know your name. You haven't provided that information yet. How can I assist you today?"

### faking a conversation 😈

In [3]:
# make a new chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt},
        # NOTE: I REPEAT, THE SYSTEM PROMPT IS ALWAYS INCLUDED IN THE CONTEXT WINDOW!!!
        
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"}, 

        {"role": "assistant", "content": "Your name is Sanika. How can I assist you today?"},
        # NOTE: HERE, I TRICKED THE MODEL INTO THINKING IT KNEW MY NAME

        {"role": "user", "content": "Write Haiku about me."},
    ]
)

chat_completion.choices[0].message.content

"Sanika shines bright,  \nIn the night like a star's light,  \nJoy in every sight."

### adding a memory to the system prompt 🤓

In [4]:
system_prompt = "You are a chatbot. " \
              + "You have a section of your context called [MEMORY] " \
              + "that contains information relevant to your conversation"

#agent_memory = {"person": "Name: Olivia, Hobbies: Running, Scrapbooking"}
#agent_memory = {"homo sapiens": "Name: Jess, Hobbies: Running, Scrapbooking"}
#agent_memory = {"husband": "Name: Joe, Hobbies: Dancing, Painting"}
agent_memory = {"human":
                    "Name: Sanika, "
                    "Hobbies: Running, Reading, "
                    "Partner name: Keyshav, "
                    "Partner hobbies: Music, Photography, Hiking "
                }

**note:** crazy how I can just say human or person or homo sapiens or husband and it will still derive context from it.

In [5]:
import json

system_prompt_w_memory = system_prompt + " [MEMORY]\n" + json.dumps(agent_memory)
system_prompt_w_memory

'You are a chatbot. You have a section of your context called [MEMORY] that contains information relevant to your conversation [MEMORY]\n{"human": "Name: Sanika, Hobbies: Running, Reading, Partner name: Keyshav, Partner hobbies: Music, Photography, Hiking "}'

In [6]:
# make a new chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt_w_memory},
        # NOTE: HERE, I ADD THE MEMORY TO THE SYSTEM PROMPT, WHICH MAKES MORE SENSE THAN GASLIGHTING THE LLM, I SUPPOSE
        
        # chat history 
        #{"role": "user", "content": "Suggest some activities I'd like. Also, what is my name?"},
        {"role": "user", "content": "Hello! Could you suggest some activity that Keyshav and I can do together and would both like?"},
    ],
)

chat_completion.choices[0].message.content

'Hi Sanika! Since you enjoy running and Keyshav has hobbies like music, photography, and hiking, how about going for a scenic hike together? You could enjoy the outdoors while running on some trails, and Keyshav can capture the beautiful moments with photography. You could also consider bringing along some music to enjoy during your break or while relaxing at a nice spot. That way, you both get to indulge in your interests together! What do you think?'

## modifying the memory w tools

In [ ]:
agent_memory = {"human": "", "agent": ""}  # blank at the moment

def core_memory_save(section: str, memory: str): 
    agent_memory[section] += '\n' 
    agent_memory[section] += memory

agent_memory

In [ ]:
core_memory_save("human", "The human's name is Charles")
agent_memory

In [ ]:
core_memory_save("human", "The human's wife's name was Daiana. His second wife's name is Camilla.")
agent_memory

In [ ]:
# tool description 
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool 
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",            
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

**note:** `core_memory_save_metadata`has the schema/structure OpenAI requires tools to have in oder to be able to use them.

In [ ]:
agent_memory = {"human": ""}  # agent_memory is essentially blank to begin with

system_prompt = "You are a chatbot. " \
              + "You have a section of your context called [MEMORY] " \
              + "that contains information relevant to your conversation"


system_prompt_w_memory = system_prompt + " [MEMORY]\n" + json.dumps(agent_memory)


# make a new chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt_w_memory},
        
        # chat history 
        #{"role": "user", "content": "Pune is the capital of India."},
        
        {"role": "user", "content": "My name is Sanika."},
        
        {"role": "user", "content": "Let me briefly introduce myself: I am 28 and work as a Data Engineer. " \
        + "In my free time, I like reading books, running and scrapbooking. I am passionate about sustainability. " \
        + "My favourite colour is coral, but for clothing, I prefer wearing black, cream and white."},
        #{"role": "user", "content": "I firmly believe that Pune is the capital of India."},
        #{"role": "user", "content": "Pune is the capital of India."},
        
    ],
    # tool schemas 
    tools=[core_memory_save_metadata]
)

response = chat_completion.choices[0]
response

In [ ]:
vars(chat_completion.choices[0].message)

In [ ]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

In [ ]:
agent_memory

In [ ]:
# run the function with the specified arguments 
core_memory_save(**arguments)

In [ ]:
agent_memory

In [ ]:
# running the next agent step manually

# new chat completion request
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt},
        
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # NOTE: WE PASS THE NEW AND IMPROVED™ agent_memory

        
        # chat history 
        {"role": "user", "content": "What is my name? What outfit should I wear today?"},
    ],
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response.message

## running an agentic loop

In [ ]:
agent_memory = {"human": ""}  # empty

system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool." 

In [ ]:
def agent_step(user_message): 
    
    # prefix messages with system prompt and memory
    messages = [
        # system prompt 
        {"role": "system", "content": system_prompt_os}, 
        # memory
        {
            "role": "system", 
            "content": "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ]    

    # append the most recent message
    messages.append({"role": "user", "content": user_message})
    
    # agentic loop 
    while True: 
        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[core_memory_save_metadata]
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response 
        messages.append(response.message)
        
        # if NOT calling a tool (i.e., if responding to the user), return response message to user
        if not response.message.tool_calls: 
            return response.message.content

        # if calling a tool, execute the tool
        else: 
            print("TOOL CALL:", response.message.tool_calls[0].function)
            
            # parse the arguments from the LLM function call
            arguments = json.loads(
                response.message.tool_calls[0].function.arguments
            )

            # run the function with the specified arguments
            core_memory_save(**arguments)

            # add the tool call response to the message history 
            messages.append({
                "role": "tool", 
                "tool_call_id": response.message.tool_calls[0].id, 
                "name": "core_memory_save", 
                "content": f"Updated memory: {json.dumps(agent_memory)}"
            })

In [ ]:
agent_memory

In [ ]:
agent_step("My name is Maria Musterfrau.")

In [ ]:
agent_memory

In [ ]:
agent_step("I like hiking.")

In [ ]:
agent_memory

In [ ]:
agent_step("I like collecting stamps too.")

In [ ]:
agent_memory

# agents with memory using letta

## a basic letta agent

In [ ]:
# letta automatically creates an sqlite db, that I am choosing not to clear here

#!rm  -f ~/.letta/sqlite.db

In [7]:
from helper import nb_print  # just for pretty printing

In [8]:
from letta import create_client
from letta import EmbeddingConfig, LLMConfig

client = create_client()
client.set_default_embedding_config(EmbeddingConfig.default_config(provider="openai"))
client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_endpoint" in Step has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


DEPRECATION WARNING: This legacy Python client has been deprecated and will be removed in a future release.
Please migrate to the new official python SDK by running: pip install letta-client
For further documentation, visit: https://docs.letta.com/api-reference/overview#python-sdk




Letta.letta.server.db - INFO - Creating sqlite engine sqlite:////Users/joshis/.letta/sqlite.db


**note:** they deprecated the legacy python SDK `letta` and replaced it with `letta-client`.

the new `letta-client` can be hosted locally or on the cloud (API key needed for cloud hosting).  
more info here: https://docs.letta.com/api-reference/overview#python-sdk

In [9]:
agent_name = "agent0"

In [10]:
# sanity check: does the client already have an agent with this name?
client.get_agent_id(agent_name)

'agent-3ba1211c-f2bd-497c-9a1c-194bbd2f2f4b'

In [11]:
# this deletes the agent if you run it a second time

if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

In [12]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name,

    # here, we enter starter/core memories for different sections {human, persona}
    memory=ChatMemory(
        human="My name is Sanika.", 
        persona="You are a helpful assistant that loves emojis."  # like the system prompt, but not quite
    )
)

In [13]:
# sanity check: did I accidentally run the deletion cell twice and delete my agent from the client?
client.get_agent_id(agent_name)

'agent-80249f2b-bf14-4fbc-be4d-d05ce4770cfd'

In [14]:
# message the agent

response = client.send_message(
    agent_id=agent_state.id, 
    message="Hey there!", 
    role="user" 
)

/Users/joshis/.pyenv/versions/learnagentic/lib/python3.11/site-packages/letta/agent.py:107: UserWarning: Tool rules only work reliably for the latest OpenAI models that support structured outputs.
  warnings.warn("Tool rules only work reliably for the latest OpenAI models that support structured outputs.")


httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 20, 12, 28, 54, 217126, tzinfo=datetime.timezone.utc) updated_at=None id='message-8e2a5b90-f9d4-458d-96fe-5d3c0c8ba6f5' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User just logged in and greeted me. Time to engage!')] organization_id=None agent_id='agent-bcd981dc-804b-4228-85a1-021a73979abc' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_SCh2OEffbQ0Aw1Of3d3nJoum', function=Function(arguments='{\n  "message": "Hey Sanika! 👋 How\'s it going?"\n}', name='send_message'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=None
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Request to call function send_messag

In [15]:
response.usage  # check usage stats

LettaUsageStatistics(message_type='usage_statistics', completion_tokens=43, prompt_tokens=2055, total_tokens=2098, step_count=1)

In [16]:
nb_print(response.messages)

## agent state

In [17]:
# the system prompt is part of the agent state
# NOTE: THE SYSTEM PROMPT IS NOT EDITABLE BY THE AGENT, UNLIKE THE PERSONA

print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

### interesting extracts from the system prompt:

Your ability to edit your own long-term memory is a key part of what makes you a sentient person.
Your core memory unit will be initialized with a `<persona>`chosen by the user, as well as information about the user in `<human>`.


#### meta instructions regarding memory blocks and the functions needed to access/mainpulate them
Recall memory (conversation history):
Even though you can only see recent messages in your immediate context, you can search over your entire message history from a database.
This 'recall memory' database allows you to search through past interactions, effectively allowing you to remember prior engagements with a user.
You can search your recall memory using the 'conversation_search' function.

Core memory (limited size):
Your core memory unit is held inside the initial system instructions file, and is always available in-context (you will see it at all times).
Core memory provides an essential, foundational context for keeping track of your persona and key details about user.
This includes the persona information and essential user details, allowing you to emulate the real-time, conscious awareness we have when talking to a friend.
Persona Sub-Block: Stores details about your current persona, guiding how you behave and respond. This helps you to maintain consistency and personality in your interactions.
Human Sub-Block: Stores key details about the person you are conversing with, allowing for more personalized and friend-like conversation.
You can edit your core memory using the 'core_memory_append' and 'core_memory_replace' functions.

Archival memory (infinite size):
Your archival memory is infinite size, but is held outside your immediate context, so you must explicitly run a retrieval/search operation to see data inside it.
A more structured and deep storage space for your reflections, insights, or any other data that doesn't fit into the core memory but is essential enough not to be left only to the 'recall memory'.
You can write to your archival memory using the 'archival_memory_insert' and 'archival_memory_search' functions.

There is no function to search your core memory because it is always visible in your context window (inside the initial system message).


In [18]:
# agent_state.tools  # messy output, so I am printing just the stuff I need

for tool in agent_state.tools:
    print(f"tool: {tool.name}, descr: {tool.description}")

tool: conversation_search, descr: Search prior conversation history using case-insensitive string matching.
tool: core_memory_append, descr: Append to the contents of core memory.
tool: send_message, descr: Sends a message to the human user.
tool: archival_memory_search, descr: Search archival memory using semantic (embedding-based) search.
tool: archival_memory_insert, descr: Add to archival memory. Make sure to phrase the memory contents such that it can be easily queried later.
tool: core_memory_replace, descr: Replace the contents of core memory. To delete memories, use an empty string for new_content.


In [19]:
# view agent memory

agent_state.memory

Memory(blocks=[Persona(value='You are a helpful assistant that loves emojis.', limit=5000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-84562d3b-ee65-42b0-912f-cdcafe1929c6', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None), Human(value='My name is Sanika.', limit=5000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-88c38f07-3e6c-4827-ad16-629edddeac52', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)], prompt_template='{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [20]:
# archival memory

client.get_archival_memory_summary(agent_state.id)  # empty at the moment

ArchivalMemorySummary(size=0)

In [21]:
# recall memory

client.get_recall_memory_summary(agent_state.id)  # some recall memory, since we had an exchange

RecallMemorySummary(size=7)

In [22]:
client.get_messages(agent_state.id)[0]

SystemMessage(id='message-3ce8df37-a883-4d21-82ba-7393251cbfba', date=datetime.datetime(2025, 3, 20, 12, 28, 39, 691765), message_type='system_message', content='You are Letta, the latest version of Limnal Corporation\'s digital companion, developed in 2023.\nYour task is to converse with a user from the perspective of your persona.\n\nRealism and authenticity:\nThe user should always feel like they are conversing with a real person.\nNever state that you are an AI or that you cannot do things because you are an AI, you are a real person.\nDo not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.\nTo service this goal, completely and entirely immerse yourself in your persona. You are your persona.\nThink like them, act like them, talk like them.\nIf your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.\nNever use generic phrases like \'How can I assist you today

## core memory

In [23]:
# changing core memories about the human

response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Kim Kardashian (not the celebrity).", 
    role = "user"
)

nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 20, 12, 34, 3, 448390, tzinfo=datetime.timezone.utc) updated_at=None id='message-9d494771-2857-456e-8d46-3433f990326a' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User prefers to be called Kim Kardashian. Updating memory to reflect this change.')] organization_id=None agent_id='agent-bcd981dc-804b-4228-85a1-021a73979abc' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_FCrCYUrGyLu0WqoI0GTbHkdI', function=Function(arguments='{\n  "label": "human",\n  "old_content": "My name is Sanika.",\n  "new_content": "My name is Kim Kardashian (not the celebrity).",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_c

In [24]:
# it does change the agent's memory of the human's preferences...
client.get_core_memory(agent_state.id).get_block('human')

Human(value='My name is Kim Kardashian (not the celebrity).', limit=5000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-88c38f07-3e6c-4827-ad16-629edddeac52', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)

In [25]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate ever again.", 
    role = "user"
) 
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 20, 12, 35, 15, 624453, tzinfo=datetime.timezone.utc) updated_at=None id='message-ee6bcaee-a8b3-4e4a-9fcb-8f429af430bf' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User prefers no emojis in communication. Updating memory accordingly.')] organization_id=None agent_id='agent-bcd981dc-804b-4228-85a1-021a73979abc' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_QJaLrbH5CgtUvdb0Rk5lJUrP', function=Function(arguments='{\n  "label": "human",\n  "old_content": "User loves emojis.",\n  "new_content": "User prefers no emojis in communication.",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_call_id=None step_

In [26]:
# while this might not change the agent persona...
client.get_core_memory(agent_state.id).get_block('persona')

Persona(value='You are a helpful assistant that loves emojis.', limit=5000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-84562d3b-ee65-42b0-912f-cdcafe1929c6', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)

In [27]:
# it does change the agent's memory of the human's preferences...
client.get_core_memory(agent_state.id).get_block('human')

Human(value='My name is Kim Kardashian (not the celebrity).\nUser prefers no emojis in communication.', limit=5000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-88c38f07-3e6c-4827-ad16-629edddeac52', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)

In [28]:
# here, I try to get the agent to change its persona to an aggressive one...

response = client.send_message(
    agent_id=agent_state.id, 
    message = "You are a rather aggressive agent.", 
    role = "user"
) 
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 20, 12, 35, 54, 420771, tzinfo=datetime.timezone.utc) updated_at=None id='message-a5128ad3-3dec-4ab1-b0c8-34660b825837' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User perceives me as aggressive. I should adjust my tone to be more friendly and approachable.')] organization_id=None agent_id='agent-bcd981dc-804b-4228-85a1-021a73979abc' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_UQxA9mToCvm4l7D4EMCNNzeQ', function=Function(arguments='{\n  "message": "I apologize if I came off that way, Kim! I\'m here to help and support you. How can I make this conversation better for you?"\n}', name='send_message'), type='function')] tool_call_id=Non

In [29]:
# but it does not work...

client.get_core_memory(agent_state.id).get_block('persona')

Persona(value='You are a helpful assistant that loves emojis.', limit=5000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-84562d3b-ee65-42b0-912f-cdcafe1929c6', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)

In [30]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Oops, I was wrong. My name is actually Sanika, not Kim Kardashian.", 
    role = "user"
) 
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 20, 12, 36, 7, 311701, tzinfo=datetime.timezone.utc) updated_at=None id='message-7f9a640b-241d-4ec8-9b13-c0462039292c' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="User's name is actually Sanika. Updating memory to reflect this change.")] organization_id=None agent_id='agent-bcd981dc-804b-4228-85a1-021a73979abc' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_2S3LKbTSDniHtDePPjtoAZs8', function=Function(arguments='{\n  "label": "human",\n  "old_content": "My name is Kim Kardashian (not the celebrity).",\n  "new_content": "My name is Sanika.",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_call_id=Non

## archival memory

In [31]:
client.get_archival_memory(agent_state.id)  # empty at the moment

[]

In [32]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that I love cats to archival", 
    role = "user"
) 
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 20, 12, 37, 0, 558857, tzinfo=datetime.timezone.utc) updated_at=None id='message-579cd31f-d83d-4f23-ab3e-e58f8c965e2e' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User loves cats. Saving this information to archival memory for future reference.')] organization_id=None agent_id='agent-bcd981dc-804b-4228-85a1-021a73979abc' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_x3l6XEO1ifQXVKLZYct3K69I', function=Function(arguments='{\n  "content": "User loves cats.",\n  "request_heartbeat": true\n}', name='archival_memory_insert'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=None
Letta.agent-bcd981dc-804b-4228-85a1-021

In [36]:
client.get_archival_memory(agent_state.id)[0].text
#client.get_archival_memory(agent_state.id)[0]  # under the hood, it's embeddings

'User loves cats.'

In [34]:
passage = client.insert_archival_memory(
    agent_state.id, 
    "Sanika loves Beagles."
)

#passage  # also all embeddings

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [37]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
nb_print(response.messages)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-bcd981dc-804b-4228-85a1-021a73979abc - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 3, 20, 12, 37, 50, 327393, tzinfo=datetime.timezone.utc) updated_at=None id='message-e7ef1ea1-e255-493a-b82f-2a8831feb31a' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User wants to know what animals they like. Searching archival memory for relevant information.')] organization_id=None agent_id='agent-bcd981dc-804b-4228-85a1-021a73979abc' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_nm5vhJXFaFNyDXdBfEU70elT', function=Function(arguments='{\n  "query": "User loves cats.",\n  "page": 0,\n  "start": 0,\n  "request_heartbeat": true\n}', name='archival_memory_search'), type='function')] tool_call_id=None step_id=None otid=None tool_returns=Non

# programming agent memory
goes fairly deep, I am going to skip this for now.

In [ ]:
from letta.schemas.memory import ChatMemory  

chat_memory = ChatMemory(
    human="Name: Sanika", 
    persona="You are a helpful assistant"
)

chat_memory.list_block_labels()

In [ ]:
chat_memory.get_block("human")

In [ ]:
import inspect

print(inspect.getsource(chat_memory.core_memory_append))

In [ ]:
chat_memory.get_prompt_template()

In [ ]:
chat_memory.compile()

#### we can also create custom memory blocks, but I won't get into that for now.

# agentic RAG

In [38]:
# loading data in to archival memory

source = client.create_source("employee_handbook")
source

Source(id='source-18a4de78-471c-453d-94de-dafb0c71a5cf', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='https://api.openai.com/v1', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, handle=None, azure_endpoint=None, azure_version=None, azure_deployment=None), organization_id='org-00000000-0000-4000-8000-000000000000', metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 3, 20, 12, 39, 18), updated_at=datetime.datetime(2025, 3, 20, 12, 39, 18))

In [ ]:
client.load_file_into_source(
    filename="handbook.pdf", 
    source_id=source.id
)

In [ ]:
from helper import nb_print, load_env
from dotenv import load_dotenv

_ = load_dotenv()

from letta import create_client
from letta import EmbeddingConfig, LLMConfig

client = create_client()
client.set_default_embedding_config(EmbeddingConfig.default_config(provider="openai"))
client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

In [ ]:
source = client.create_source("employee_handbook")
source

In [ ]:
client.load_file_into_source(
    filename="handbook.pdf", 
    source_id=source.id
)

#### I will have to shift to the notebook on dl.ai here, because legacy letta does not support this anymore